In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.append('/home/saptarshi/Dhruv/Dissertation/ICC/')
from generate_attributions import *
from utils import CONFIG,compute_pgu_pgi_sums,evaluate_exp,PREFIX,compute_auc,compute_pgu_pgi_sums

/home/saptarshi/anaconda3/envs/Dhruv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## German Credit Dataset

In [6]:
config = CONFIG['german']
mlp_path = PREFIX + 'models/german_mlp/german_mlp_seed_10/logs/version_1/checkpoints/epoch=62-step=819.ckpt'
scm_path = PREFIX + 'models/german_nf/german_nf_seed_20/logs/version_0/checkpoints/epoch=509-step=6630.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [ ]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [9]:
# Load JSON
with open(PREFIX + "attributions/german/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [10]:
ge_dict = data["ge_dict"]
evaluation_metrics = evaluate_exp(ge_dict,config,mlp_model)
print(evaluation_metrics)
# Display the results
agg_scores = compute_pgu_pgi_sums(evaluation_metrics)
for method, scores in agg_scores.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")


{'ig': {'pgi': {'k=1': [0.2710515260696411], 'k=2': [0.38441479206085205], 'k=3': [0.4239215552806854], 'k=4': [0.3952651619911194]}, 'pgu': {'k=1': [0.3510115444660187], 'k=2': [0.23916801810264587], 'k=3': [0.13131259381771088], 'k=4': [1.4081970256540899e-08]}}, 'itg': {'pgi': {'k=1': [0.22782570123672485], 'k=2': [0.23931904137134552], 'k=3': [0.35263949632644653], 'k=4': [0.3949895203113556]}, 'pgu': {'k=1': [0.3647838234901428], 'k=2': [0.3855058550834656], 'k=3': [0.27076393365859985], 'k=4': [1.4081970256540899e-08]}}, 'sg': {'pgi': {'k=1': [0.27131354808807373], 'k=2': [0.3362196683883667], 'k=3': [0.3156304955482483], 'k=4': [0.3953288793563843]}, 'pgu': {'k=1': [0.3524555563926697], 'k=2': [0.2752402126789093], 'k=3': [0.24017396569252014], 'k=4': [1.4081970256540899e-08]}}, 'shap': {'pgi': {'k=1': [0.27070534229278564], 'k=2': [0.27204689383506775], 'k=3': [0.3160041868686676], 'k=4': [0.3943582773208618]}, 'pgu': {'k=1': [0.3518504798412323], 'k=2': [0.3693719506263733], '

In [14]:
auc_results = compute_auc(evaluation_metrics,config)

# Print results nicely
for method, vals in auc_results.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']:.6f}, PGU AUC = {vals['pgu_auc']:.6f}")


ig: PGI AUC = 0.380498, PGU AUC = 0.181995
itg: PGI AUC = 0.301122, PGU AUC = 0.279554
sg: PGI AUC = 0.328390, PGU AUC = 0.230547
shap: PGI AUC = 0.306861, PGU AUC = 0.261699
lime: PGI AUC = 0.328377, PGU AUC = 0.230698
sp_lime: PGI AUC = 0.285448, PGU AUC = 0.289359
pfi: PGI AUC = 0.328246, PGU AUC = 0.230716
icc_topo: PGI AUC = 0.307319, PGU AUC = 0.261849
icc_shap: PGI AUC = 0.307153, PGU AUC = 0.261956


## Lung Cancer Dataset

In [19]:
config = CONFIG['cancer']
mlp_path = PREFIX + 'models/cancer_mlp/cancer_mlp_seed_20/logs/version_0/checkpoints/epoch=62-step=2394.ckpt'
scm_path = PREFIX + 'models/cancer_nf/cancer_nf_seed_10/logs/version_0/checkpoints/epoch=203-step=7752.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [ ]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

In [20]:
# Load JSON
with open(PREFIX +"attributions/cancer/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [21]:
# Call the function
ge_dict = data["ge_dict"]
evaluation_metrics = evaluate_exp(ge_dict,config,mlp_model)
print(evaluation_metrics)
# Display the results
agg_scores = compute_pgu_pgi_sums(evaluation_metrics)
for method, scores in agg_scores.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")

{'ig': {'pgi': {'k=1': [0.29632842540740967], 'k=2': [0.3059897720813751], 'k=3': [0.34692874550819397], 'k=4': [0.4179578423500061], 'k=5': [0.4467124044895172], 'k=6': [0.4612525999546051], 'k=7': [0.46797043085098267]}, 'pgu': {'k=1': [0.4187425673007965], 'k=2': [0.4037628173828125], 'k=3': [0.37662848830223083], 'k=4': [0.30643293261528015], 'k=5': [0.2416532337665558], 'k=6': [0.13601255416870117], 'k=7': [1.783449299352924e-08]}}, 'itg': {'pgi': {'k=1': [0.14735865592956543], 'k=2': [0.30135613679885864], 'k=3': [0.3315041661262512], 'k=4': [0.3526589572429657], 'k=5': [0.39535462856292725], 'k=6': [0.419208824634552], 'k=7': [0.467991441488266]}, 'pgu': {'k=1': [0.45067164301872253], 'k=2': [0.4137524664402008], 'k=3': [0.3870377242565155], 'k=4': [0.3780052661895752], 'k=5': [0.335883766412735], 'k=6': [0.2964792549610138], 'k=7': [1.783449299352924e-08]}}, 'sg': {'pgi': {'k=1': [0.214140847325325], 'k=2': [0.30123406648635864], 'k=3': [0.33047375082969666], 'k=4': [0.37680202

In [22]:
auc_results = compute_auc(evaluation_metrics,config)
for method, vals in auc_results.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']:.6f}, PGU AUC = {vals['pgu_auc']:.6f}")


ig: PGI AUC = 0.393498, PGU AUC = 0.278977
itg: PGI AUC = 0.351293, PGU AUC = 0.339416
sg: PGI AUC = 0.360866, PGU AUC = 0.333498
shap: PGI AUC = 0.418459, PGU AUC = 0.271413
lime: PGI AUC = 0.395363, PGU AUC = 0.293764
sp_lime: PGI AUC = 0.349963, PGU AUC = 0.311630
pfi: PGI AUC = 0.417300, PGU AUC = 0.279413
icc_topo: PGI AUC = 0.413235, PGU AUC = 0.273519
icc_shap: PGI AUC = 0.415082, PGU AUC = 0.271799


## Synthetic Data

In [2]:
config = CONFIG['syn']
mlp_path = PREFIX + 'models/syn_mlp/syn_mlp_seed_20/logs/version_3/checkpoints/epoch=12-step=416.ckpt'
scm_path = PREFIX + 'models/syn_nf/syn_nf_seed_20/logs/version_1/checkpoints/epoch=3-step=128.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [9]:
# Load JSON
with open(PREFIX +"attributions/syn/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [10]:
# Call the function
ge_dict = data["ge_dict"]
evaluation_metrics = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)
# Display the results
agg_scores = compute_pgu_pgi_sums(evaluation_metrics)
for method, scores in agg_scores.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")

{'ig': {'pgi': {'k=1': [0.029777446761727333], 'k=2': [0.04713110998272896], 'k=3': [0.06602025032043457]}, 'pgu': {'k=1': [0.044813379645347595], 'k=2': [0.024564575403928757], 'k=3': [1.8179417082819782e-08]}}, 'itg': {'pgi': {'k=1': [0.02531259134411812], 'k=2': [0.044942617416381836], 'k=3': [0.0658622682094574]}, 'pgu': {'k=1': [0.04934872314333916], 'k=2': [0.029689406976103783], 'k=3': [1.8179417082819782e-08]}}, 'sg': {'pgi': {'k=1': [0.029701614752411842], 'k=2': [0.047080982476472855], 'k=3': [0.06585939228534698]}, 'pgu': {'k=1': [0.04492420703172684], 'k=2': [0.024594595655798912], 'k=3': [1.8179417082819782e-08]}}, 'shap': {'pgi': {'k=1': [0.029781628400087357], 'k=2': [0.047010328620672226], 'k=3': [0.06585140526294708]}, 'pgu': {'k=1': [0.04497193172574043], 'k=2': [0.024589834734797478], 'k=3': [1.8179417082819782e-08]}}, 'lime': {'pgi': {'k=1': [0.029750090092420578], 'k=2': [0.0470273420214653], 'k=3': [0.06585081666707993]}, 'pgu': {'k=1': [0.04483410343527794], 'k=2

In [11]:
auc_results = compute_auc(evaluation_metrics,config)
for method, vals in auc_results.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']:.6f}, PGU AUC = {vals['pgu_auc']:.6f}")


ig: PGI AUC = 0.047515, PGU AUC = 0.023486
itg: PGI AUC = 0.045265, PGU AUC = 0.027182
sg: PGI AUC = 0.047431, PGU AUC = 0.023528
shap: PGI AUC = 0.047413, PGU AUC = 0.023538
lime: PGI AUC = 0.047414, PGU AUC = 0.023481
sp_lime: PGI AUC = 0.047329, PGU AUC = 0.024421
pfi: PGI AUC = 0.048606, PGU AUC = 0.023838
icc_topo: PGI AUC = 0.047459, PGU AUC = 0.023541
icc_shap: PGI AUC = 0.047382, PGU AUC = 0.023481


## Test

In [2]:
config = CONFIG['syn']
mlp_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_ann/syn_ann_seed_10/logs/version_0/checkpoints/epoch=161-step=5184.ckpt'
scm_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_nf/syn_nf_seed_10/logs/version_0/checkpoints/epoch=19-step=640.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_ann'],classification=False))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))
ge_dict, evaluation_metrics, time_dict, global_explanations = generate_global_exps(config,mlp_model,scm_model)

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [3]:
ge_dict

{'ig': {'W': [0.008685602410414437],
  'Z': [0.5541948194138167],
  'X': [0.4371195781757688]},
 'itg': {'W': [0.008485306748260027],
  'Z': [0.497316907775009],
  'X': [0.49419778547673104]},
 'sg': {'W': [0.13136424456458626],
  'Z': [0.3860659244300918],
  'X': [0.48256983100532197]},
 'shap': {'W': [0.5803365821763931],
  'Z': [0.3543153621348831],
  'X': [0.06534805568872387]},
 'lime': {'W': [0.2837432524861816],
  'Z': [0.293367905107146],
  'X': [0.4228888424066725]},
 'sp_lime': {'X': [0.41602805243460733],
  'W': [0.2947343644233361],
  'Z': [0.2892375831420565]},
 'pfi': {'W': [0.2555289566516876],
  'Z': [0.2728649377822876],
  'X': [0.47160604596138]},
 'icc_topo': {'W': [0.07992755621671677],
  'Z': [0.6083893179893494],
  'X': [0.31013962626457214]},
 'icc_shap': {'W': [0.1287594437599182],
  'Z': [0.6630550622940063],
  'X': [0.20664207637310028]}}

In [9]:
generate_attr_plot(global_explanations,config)

In [7]:
# Load JSON
with open("/home/dhruv/Files/Thesis/Dissertation/Code/attributions/syn/attribution.json", "r") as f:
    data = json.load(f)

# Call the function
summary = compute_pgu_pgi_sums(data["evaluation_metrics"])

# Display the results
for method, scores in summary.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")


ig: PGU Sum = 0.128419, PGI Sum = 0.000436
itg: PGU Sum = 0.128419, PGI Sum = 0.000436
sg: PGU Sum = 0.128418, PGI Sum = 0.000437
shap: PGU Sum = 0.128420, PGI Sum = 0.000435
lime: PGU Sum = 0.128418, PGI Sum = 0.000437
sp_lime: PGU Sum = 0.128420, PGI Sum = 0.000435
pfi: PGU Sum = 0.128418, PGI Sum = 0.000437
icc_topo: PGU Sum = 0.128419, PGI Sum = 0.000436
icc_shap: PGU Sum = 0.128419, PGI Sum = 0.000436
